In [115]:
import pandas as pd
import numpy as np
import os
import src.DataWrangling as DataWrangling
from pathlib import Path
from hmmlearn.hmm import GaussianHMM
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
PROJECT_ROOT = Path("..").resolve()
PROJECT_ROOT


PosixPath('/home/youssof/Desktop/Sepsis')

In [2]:
measurement_meds = pd.read_csv(f"{PROJECT_ROOT}/data/cleaned_training/measurement_meds.csv")
measurement_lab = pd.read_csv(f"{PROJECT_ROOT}/data/cleaned_training/measurement_lab.csv")

In [9]:
measurement_meds.head()

,visit_occurrence_id,person_id,measurement_datetime,Systolic blood pressure,Diastolic blood pressure,Body temperature,Respiratory rate,Heart rate,Measurement of oxygen saturation at periphery,Oxygen/Gas total [Pure volume fraction] Inhaled gas
0,2041654295,2255421,2019-10-04 10:00:00,NaN,NaN,37.5,NaN,NaN,NaN,NaN
1,2041654295,2255421,2019-10-04 12:00:00,NaN,NaN,37.0,NaN,NaN,NaN,NaN
2,2041654295,2255421,2019-10-04 14:00:00,NaN,NaN,36.7,NaN,NaN,NaN,NaN
3,2041654295,2255421,2019-10-04 16:00:00,NaN,NaN,37.0,NaN,NaN,NaN,NaN
4,2041654295,2255421,2019-10-04 18:00:00,NaN,NaN,37.0,NaN,NaN,NaN,NaN


In [10]:
measurement_lab.head()

,visit_occurrence_id,person_id,measurement_datetime,Base excess in Venous blood by calculation,Base excess in Arterial blood by calculation,Phosphate [Moles/volume] in Serum or Plasma,Potassium [Moles/volume] in Blood,Bilirubin.total [Moles/volume] in Serum or Plasma,Neutrophil Ab [Units/volume] in Serum,Bicarbonate [Moles/volume] in Arterial blood,...,Partial thromboplastin time,activated,Total white blood count,Platelet count,White blood cell count,Blood venous pH,D-dimer level,Blood arterial pH,Hemoglobin [Moles/volume] in Blood,Ionised calcium measurement
0,59452488,510305,2019-07-01 16:00:00,-3.4,NaN,NaN,3.72,16.5,2.5,NaN,...,31.9,4.0,44.0,0.6,7.42,NaN,NaN,8.6,1.25,NaN
1,59452488,510305,2019-07-01 17:00:00,-0.3,NaN,NaN,4.06,16.5,2.5,NaN,...,31.9,4.3,41.0,0.6,7.53,NaN,NaN,8.9,1.25,NaN
2,59452488,510305,2019-07-02 13:00:00,-0.3,NaN,NaN,4.06,16.5,2.6,NaN,...,31.9,4.3,41.0,0.0,7.53,NaN,NaN,8.9,1.25,NaN
3,59452488,510305,2019-07-03 09:00:00,-0.3,NaN,NaN,4.06,16.5,2.6,NaN,...,31.9,4.3,41.0,0.0,7.53,NaN,NaN,8.9,1.25,NaN
4,59452488,510305,2019-07-03 10:00:00,-0.3,NaN,NaN,4.06,16.5,2.6,NaN,...,31.9,5.1,13.6,0.0,7.53,NaN,NaN,7.1,1.25,NaN


In [16]:
data_merged = measurement_lab.merge(measurement_meds,how="outer",on=["person_id","measurement_datetime","visit_occurrence_id"]).reset_index(drop=True)
data_merged.head()

,visit_occurrence_id,person_id,measurement_datetime,Base excess in Venous blood by calculation,Base excess in Arterial blood by calculation,Phosphate [Moles/volume] in Serum or Plasma,Potassium [Moles/volume] in Blood,Bilirubin.total [Moles/volume] in Serum or Plasma,Neutrophil Ab [Units/volume] in Serum,Bicarbonate [Moles/volume] in Arterial blood,...,Blood arterial pH,Hemoglobin [Moles/volume] in Blood,Ionised calcium measurement,Systolic blood pressure,Diastolic blood pressure,Body temperature,Respiratory rate,Heart rate,Measurement of oxygen saturation at periphery,Oxygen/Gas total [Pure volume fraction] Inhaled gas
0,59452488,510305,2019-07-01 16:00:00,-3.4,NaN,NaN,3.72,16.5,2.5,NaN,...,8.6,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59452488,510305,2019-07-01 17:00:00,-0.3,NaN,NaN,4.06,16.5,2.5,NaN,...,8.9,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59452488,510305,2019-07-02 13:00:00,-0.3,NaN,NaN,4.06,16.5,2.6,NaN,...,8.9,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,59452488,510305,2019-07-03 09:00:00,-0.3,NaN,NaN,4.06,16.5,2.6,NaN,...,8.9,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59452488,510305,2019-07-03 10:00:00,-0.3,NaN,NaN,4.06,16.5,2.6,NaN,...,7.1,1.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [194]:
features = data_merged.drop(["person_id","measurement_datetime","visit_occurrence_id"],axis=1).columns
print(len(features))

pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),

])

transformer = ColumnTransformer([
    ("num", pipeline, features),
], remainder="passthrough")

data_imputed_scaled = pd.DataFrame(transformer.fit_transform(data_merged), columns= transformer.get_feature_names_out())
data_imputed_scaled.head()

44


/home/youssof/Desktop/Sepsis/.venv/lib/python3.12/site-packages/sklearn/impute/_base.py:641: UserWarning: Skipping features without any observed values: ['Ionised calcium measurement']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


,num__Base excess in Venous blood by calculation,num__Base excess in Arterial blood by calculation,num__Phosphate [Moles/volume] in Serum or Plasma,num__Potassium [Moles/volume] in Blood,num__Bilirubin.total [Moles/volume] in Serum or Plasma,num__Neutrophil Ab [Units/volume] in Serum,num__Bicarbonate [Moles/volume] in Arterial blood,num__Hematocrit [Volume Fraction] of Blood,num__Glucose [Moles/volume] in Serum or Plasma,num__Calcium [Moles/volume] in Serum or Plasma,...,num__Systolic blood pressure,num__Diastolic blood pressure,num__Body temperature,num__Respiratory rate,num__Heart rate,num__Measurement of oxygen saturation at periphery,num__Oxygen/Gas total [Pure volume fraction] Inhaled gas,remainder__visit_occurrence_id,remainder__person_id,remainder__measurement_datetime
0,-1.515548,-0.055852,-0.031513,-0.843574,0.411574,-0.879491,-0.039119,-1.885527,-0.351855,-0.012483,...,0.000943,-0.044647,-0.012893,-0.082202,0.016799,0.167925,-0.008805,59452488,510305,2019-07-01 16:00:00
1,-0.06244,-0.055852,-0.031513,0.174889,0.411574,-0.879491,-0.039119,-1.419874,-0.351855,-0.012483,...,0.000943,-0.044647,-0.012893,-0.082202,0.016799,0.167925,-0.008805,59452488,510305,2019-07-01 17:00:00
2,-0.06244,-0.055852,-0.031513,0.174889,0.411574,-0.845734,-0.039119,-1.419874,-0.351855,-0.012483,...,0.000943,-0.044647,-0.012893,-0.082202,0.016799,0.167925,-0.008805,59452488,510305,2019-07-02 13:00:00
3,-0.06244,-0.055852,-0.031513,0.174889,0.411574,-0.845734,-0.039119,-1.419874,-0.351855,-0.012483,...,0.000943,-0.044647,-0.012893,-0.082202,0.016799,0.167925,-0.008805,59452488,510305,2019-07-03 09:00:00
4,-0.06244,-0.055852,-0.031513,0.174889,0.411574,-0.845734,-0.039119,-3.127269,-0.351855,-0.012483,...,0.000943,-0.044647,-0.012893,-0.082202,0.016799,0.167925,-0.008805,59452488,510305,2019-07-03 10:00:00


In [195]:
data_imputed_scaled.sort_values(by= ["remainder__person_id","remainder__measurement_datetime"],inplace=True)
data_imputed_scaled.head()

,num__Base excess in Venous blood by calculation,num__Base excess in Arterial blood by calculation,num__Phosphate [Moles/volume] in Serum or Plasma,num__Potassium [Moles/volume] in Blood,num__Bilirubin.total [Moles/volume] in Serum or Plasma,num__Neutrophil Ab [Units/volume] in Serum,num__Bicarbonate [Moles/volume] in Arterial blood,num__Hematocrit [Volume Fraction] of Blood,num__Glucose [Moles/volume] in Serum or Plasma,num__Calcium [Moles/volume] in Serum or Plasma,...,num__Systolic blood pressure,num__Diastolic blood pressure,num__Body temperature,num__Respiratory rate,num__Heart rate,num__Measurement of oxygen saturation at periphery,num__Oxygen/Gas total [Pure volume fraction] Inhaled gas,remainder__visit_occurrence_id,remainder__person_id,remainder__measurement_datetime
0,-1.515548,-0.055852,-0.031513,-0.843574,0.411574,-0.879491,-0.039119,-1.885527,-0.351855,-0.012483,...,0.000943,-0.044647,-0.012893,-0.082202,0.016799,0.167925,-0.008805,59452488,510305,2019-07-01 16:00:00
1,-0.06244,-0.055852,-0.031513,0.174889,0.411574,-0.879491,-0.039119,-1.419874,-0.351855,-0.012483,...,0.000943,-0.044647,-0.012893,-0.082202,0.016799,0.167925,-0.008805,59452488,510305,2019-07-01 17:00:00
2,-0.06244,-0.055852,-0.031513,0.174889,0.411574,-0.845734,-0.039119,-1.419874,-0.351855,-0.012483,...,0.000943,-0.044647,-0.012893,-0.082202,0.016799,0.167925,-0.008805,59452488,510305,2019-07-02 13:00:00
3,-0.06244,-0.055852,-0.031513,0.174889,0.411574,-0.845734,-0.039119,-1.419874,-0.351855,-0.012483,...,0.000943,-0.044647,-0.012893,-0.082202,0.016799,0.167925,-0.008805,59452488,510305,2019-07-03 09:00:00
4,-0.06244,-0.055852,-0.031513,0.174889,0.411574,-0.845734,-0.039119,-3.127269,-0.351855,-0.012483,...,0.000943,-0.044647,-0.012893,-0.082202,0.016799,0.167925,-0.008805,59452488,510305,2019-07-03 10:00:00


In [196]:
lengths = data_imputed_scaled.groupby('remainder__person_id').size().tolist()
X = data_imputed_scaled.drop(["remainder__person_id","remainder__measurement_datetime","remainder__visit_occurrence_id"],axis=1).to_numpy()
gauss_hmm = GaussianHMM(n_components=4, covariance_type="diag", n_iter=200)
gauss_hmm.fit(X,lengths=lengths)
lengths

[17,
 41,
 46,
 191,
 18,
 13,
 59,
 42,
 34,
 29,
 1624,
 165,
 89,
 28,
 68,
 54,
 143,
 3,
 130,
 28,
 56,
 15,
 79,
 314,
 82,
 36,
 29,
 11,
 63,
 189,
 87,
 300,
 34,
 41,
 115,
 5,
 123,
 58,
 28,
 38,
 651,
 28,
 312,
 16,
 22,
 10,
 19,
 120,
 62,
 90,
 16,
 25,
 39,
 87,
 40,
 1766,
 37,
 139,
 31,
 27,
 27,
 243,
 63,
 30,
 36,
 340,
 24,
 45,
 16,
 19,
 228,
 136,
 17,
 2,
 11,
 8,
 21,
 28,
 37,
 21,
 177,
 70,
 47,
 14,
 13,
 32,
 149,
 33,
 27,
 819,
 46,
 48,
 60,
 25,
 71,
 18,
 331,
 106,
 33,
 912,
 21,
 140,
 90,
 140,
 56,
 48,
 99,
 26,
 102,
 492,
 52,
 12,
 95,
 60,
 17,
 59,
 31,
 48,
 94,
 16,
 45,
 26,
 50,
 20,
 23,
 154,
 737,
 25,
 56,
 23,
 923,
 140,
 29,
 63,
 16,
 21,
 24,
 89,
 107,
 61,
 27,
 150,
 21,
 67,
 21,
 25,
 104,
 53,
 126,
 140,
 43,
 149,
 52,
 66,
 671,
 94,
 228,
 17,
 22,
 32,
 33,
 69,
 85,
 85,
 34,
 90,
 24,
 103,
 37,
 24,
 130,
 75,
 361,
 84,
 4,
 15,
 25,
 21,
 10,
 27,
 64,
 289,
 3,
 57,
 5,
 23,
 291,
 173,
 202,
 40,
 109,
 

In [35]:
gauss_hmm.transmat_

array([[0.9137556 , 0.00918047, 0.01485763, 0.0622063 ],
       [0.4296661 , 0.49985478, 0.00565452, 0.0648246 ],
       [0.62049822, 0.00341098, 0.32854225, 0.04754855],
       [0.25984857, 0.00525101, 0.00793932, 0.72696111]])

In [36]:
probabilities = gauss_hmm.predict_proba(X, lengths=lengths)
pd.DataFrame(probabilities)

,0,1,2,3
0,0.0,3.947221e-31,6.435446e-22,1.000000e+00
1,0.0,4.843323e-31,8.842290e-22,1.000000e+00
2,0.0,4.040312e-31,1.454611e-21,1.000000e+00
3,0.0,2.381309e-31,1.309237e-21,1.000000e+00
4,0.0,4.554013e-31,1.098680e-21,1.000000e+00
...,...,...,...,...
248151,1.0,1.329837e-151,3.100982e-139,7.476814e-122
248152,1.0,2.055554e-151,5.363044e-139,2.085161e-123
248153,1.0,1.165722e-151,2.710912e-139,1.883818e-121
248154,1.0,1.288483e-151,3.412402e-139,4.876025e-122


In [43]:
sepsis_labels = DataWrangling.get_sepsis_labels(f"{PROJECT_ROOT}/data/training_data/SepsisLabel_train.csv")
non_septic_cases = sepsis_labels[sepsis_labels["SepsisLabel"] == 0]
non_septic_cases.shape

(2041, 2)

In [45]:
random_sample = non_septic_cases.sample(n = 100, random_state = 42)
new_population = pd.concat([random_sample,sepsis_labels[sepsis_labels["SepsisLabel"] == 1]],ignore_index=True)
new_population["SepsisLabel"].value_counts()

SepsisLabel
0    100
1     78
Name: count, dtype: int64

In [48]:
data_resampled = data_imputed_scaled[data_imputed_scaled["remainder__person_id"].isin(new_population["person_id"].tolist())]
data_resampled.shape

(58245, 46)

In [117]:
features = data_merged.drop(["person_id","measurement_datetime","visit_occurrence_id"],axis=1).columns
print(len(features))

pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=10)),

])

transformer = ColumnTransformer([
    ("num", pipeline, features),
], remainder="passthrough")

data_pca = pd.DataFrame(transformer.fit_transform(data_merged), columns= transformer.get_feature_names_out())
data_pca

44


/home/youssof/Desktop/Sepsis/.venv/lib/python3.12/site-packages/sklearn/impute/_base.py:641: UserWarning: Skipping features without any observed values: ['Ionised calcium measurement']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


,num__pca0,num__pca1,num__pca2,num__pca3,num__pca4,num__pca5,num__pca6,num__pca7,num__pca8,num__pca9,remainder__visit_occurrence_id,remainder__person_id,remainder__measurement_datetime
0,-1.809835,2.221654,0.318371,-3.178412,2.705674,0.787426,1.354563,-1.094227,-1.189474,-1.697096,59452488,510305,2019-07-01 16:00:00
1,-0.199778,2.393021,0.547581,-3.262662,2.860811,-0.948642,0.100277,0.914594,-3.270176,-2.048973,59452488,510305,2019-07-01 17:00:00
2,-0.260183,2.067997,0.499651,-3.402361,2.659664,-0.656788,-0.188967,1.188415,-3.482343,-2.021449,59452488,510305,2019-07-02 13:00:00
3,-0.292433,1.744817,0.509581,-3.115493,2.502801,-0.698523,-0.078974,1.772136,-3.501087,-1.725127,59452488,510305,2019-07-03 09:00:00
4,-0.490531,1.57948,0.514027,-3.432406,2.901836,-0.591044,-0.981003,1.570505,-3.403123,-1.658521,59452488,510305,2019-07-03 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
248151,-0.116743,-0.405114,-0.041453,-0.025541,-0.113493,-0.021235,-0.161735,-0.154811,-0.064563,0.053543,1464914453,2146982383,2019-12-24 01:00:00
248152,-0.100469,-0.39356,-0.092202,-0.042398,-0.073368,-0.045504,-0.164264,-0.236526,-0.217303,0.23011,1464914453,2146982383,2019-12-24 03:00:00
248153,-0.128367,-0.413366,-0.005205,-0.013501,-0.142154,-0.003899,-0.159928,-0.096443,0.044538,-0.072576,1464914453,2146982383,2019-12-24 05:00:00
248154,-0.146966,-0.426571,0.052794,0.005764,-0.18801,0.023837,-0.157038,-0.003054,0.219098,-0.274367,1464914453,2146982383,2019-12-24 11:00:00


In [119]:
data_resampled = data_pca[data_pca["remainder__person_id"].isin(new_population["person_id"].tolist())]
data_resampled.shape

(58245, 13)

In [186]:
lengths = data_resampled.groupby('remainder__person_id').size().tolist()
X = data_resampled.drop(["remainder__person_id","remainder__measurement_datetime","remainder__visit_occurrence_id"],axis=1).to_numpy()
gauss_hmm = GaussianHMM(n_components=4, covariance_type="diag", n_iter=200)
gauss_hmm.fit(X,lengths=lengths)
gauss_hmm.transmat_

array([[0.86295145, 0.02251811, 0.02346482, 0.09106562],
       [0.14080438, 0.53205502, 0.31119924, 0.01594137],
       [0.02209179, 0.04694763, 0.91222583, 0.01873475],
       [0.32317251, 0.00982656, 0.06569812, 0.60130282]])

In [187]:
probabilities = gauss_hmm.predict_proba(X, lengths=lengths)
probs = pd.DataFrame(probabilities, columns=[f"s{i}" for i in range(probabilities.shape[1])])
probs

,s0,s1,s2,s3
0,0.0,5.869107e-04,0.000000e+00,9.994131e-01
1,0.0,2.650875e-06,0.000000e+00,9.999973e-01
2,0.0,4.579647e-06,0.000000e+00,9.999954e-01
3,0.0,2.435360e-03,0.000000e+00,9.975646e-01
4,0.0,1.000000e+00,0.000000e+00,3.121611e-10
...,...,...,...,...
58240,1.0,5.654741e-23,3.531142e-12,8.862247e-19
58241,1.0,7.981808e-25,6.116816e-14,1.225344e-20
58242,1.0,4.274933e-22,2.466531e-11,6.737350e-18
58243,1.0,1.696865e-25,1.519924e-14,2.536868e-21


In [188]:
state_predictions = gauss_hmm.predict(X, lengths=lengths)
pd.Series(state_predictions).value_counts(normalize=True)

2    0.420242
0    0.400137
3    0.115237
1    0.064383
Name: proportion, dtype: float64

In [189]:
sepsis = pd.read_csv(f'{PROJECT_ROOT}/data/training_data/SepsisLabel_train.csv')
sepsis = sepsis[sepsis["person_id"].isin(new_population["person_id"].tolist())].reset_index(drop=True)

final_df = data_resampled.reset_index(drop=True).join(probs).merge(sepsis,how="left",left_on=["remainder__person_id","remainder__measurement_datetime"],right_on=["person_id","measurement_datetime"])
final_df["state"] = state_predictions
final_df

,num__pca0,num__pca1,num__pca2,num__pca3,num__pca4,num__pca5,num__pca6,num__pca7,num__pca8,num__pca9,...,remainder__person_id,remainder__measurement_datetime,s0,s1,s2,s3,person_id,measurement_datetime,SepsisLabel,state
0,-5.003658,0.745043,-0.245956,4.233388,-2.370243,4.134952,4.294899,3.644691,-2.03543,0.296808,...,16748754,2020-05-28 15:00:00,0.0,5.869107e-04,0.000000e+00,9.994131e-01,1.674875e+07,2020-05-28 15:00:00,0.0,3
1,-5.160246,1.324491,0.061534,1.999849,0.298693,2.947001,2.030549,2.26784,-0.776428,0.529852,...,16748754,2020-06-16 15:00:00,0.0,2.650875e-06,0.000000e+00,9.999973e-01,1.674875e+07,2020-06-16 15:00:00,1.0,3
2,-2.547169,-0.496302,0.040651,0.452775,-0.544995,0.896699,2.156697,2.995507,-0.143113,1.31239,...,16748754,2020-06-16 16:00:00,0.0,4.579647e-06,0.000000e+00,9.999954e-01,1.674875e+07,2020-06-16 16:00:00,1.0,3
3,-2.568046,-0.254812,0.123647,0.891136,-0.364854,0.86775,2.381676,3.785645,-0.136798,1.741695,...,16748754,2020-06-16 17:00:00,0.0,2.435360e-03,0.000000e+00,9.975646e-01,1.674875e+07,2020-06-16 17:00:00,1.0,3
4,-2.48694,2.322882,-2.636258,4.174139,-1.67678,1.201542,-1.8437,-0.105094,0.98327,2.3063,...,16748754,2020-06-16 19:00:00,0.0,1.000000e+00,0.000000e+00,3.121611e-10,1.674875e+07,2020-06-16 19:00:00,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58240,-0.144641,-0.42492,0.045544,0.003356,-0.182278,0.02037,-0.157399,-0.014728,0.197278,-0.249144,...,2137359561,2020-03-13 11:00:00,1.0,5.654741e-23,3.531142e-12,8.862247e-19,2.137360e+09,2020-03-13 11:00:00,0.0,0
58241,-0.137666,-0.419969,0.023795,-0.003868,-0.165082,0.009969,-0.158483,-0.049748,0.131818,-0.173472,...,2137359561,2020-03-13 13:00:00,1.0,7.981808e-25,6.116816e-14,1.225344e-20,2.137360e+09,2020-03-13 13:00:00,0.0,0
58242,-0.146966,-0.426571,0.052794,0.005764,-0.18801,0.023837,-0.157038,-0.003054,0.219098,-0.274367,...,2137359561,2020-03-13 16:00:00,1.0,4.274933e-22,2.466531e-11,6.737350e-18,2.137360e+09,2020-03-13 16:00:00,0.0,0
58243,-0.130692,-0.415017,0.002045,-0.011093,-0.147886,-0.000432,-0.159567,-0.084769,0.066358,-0.0978,...,2137359561,2020-03-13 18:00:00,1.0,1.696865e-25,1.519924e-14,2.536868e-21,2.137360e+09,2020-03-13 18:00:00,0.0,0


In [190]:
pd.get_dummies(final_df,columns=["SepsisLabel"],dtype=np.int8).groupby("state")[["SepsisLabel_0.0","SepsisLabel_1.0"]].sum()

,SepsisLabel_0.0,SepsisLabel_1.0
state,,
0,20370,1861
1,3091,621
2,23040,1103
3,5815,617
